In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
import nltk.stem

stemmer = nltk.stem.SnowballStemmer('english')

In [2]:
wine_df = pd.read_csv('winemag-data-130k-v2.csv')

In [7]:
class StemmedCountVectorizer(text.CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
vectorizer = StemmedCountVectorizer(analyzer="word", stop_words='english')
processed_reviews = vectorizer.fit_transform(wine_df['description'])

In [26]:
# creating new columns names for full df
new_cols = list(wine_df.columns) + list(vectorizer.vocabulary_.keys())

In [13]:
# merging wine_df and BoW
test = pd.concat([wine_df, pd.DataFrame.sparse.from_spmatrix(processed_reviews)], axis=1)

In [27]:
# renaming columns
test.columns = new_cols